#this notebook will be mainly used for the capstone project

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Week3 - Clustering Toronto's neighborhoods

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Go to the page in wikipedia

In [6]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
url=r.content
soup = BeautifulSoup(url,'html.parser')

### choosing the table from the page

In [7]:
My_table=soup.find("table",{"class":"wikitable sortable"})

table_rows = My_table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [8]:
df=pd.DataFrame(l, columns=["PostalCode", "Borough","Neighborhood"])

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [9]:
df['Neighborhood'] = df['Neighborhood'].str.replace('\n', '')

df1 = df[df.Borough != 'Not assigned']

In [10]:
print('Table head:')
print(df1.head())
print('Table shape:',df1.shape)

Table head:
  PostalCode           Borough      Neighborhood
0       None              None              None
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront
6        M6A        North York  Lawrence Heights
Table shape: (211, 3)


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page,
#### you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.
#### These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [11]:
df2=df1.groupby('PostalCode')['Neighborhood'].apply(','.join).to_frame().reset_index('PostalCode')
df2=df2.rename(columns={"Neighborhood": "Neighborhoods"})
df4=df1.drop(columns=['Neighborhood'])
df3= pd.merge(df4, df2, how='inner', left_on = 'PostalCode', right_on = 'PostalCode')

df5=df3.drop_duplicates()

print(df5.head(10))
print('Table shape:',df5.shape)

   PostalCode           Borough                    Neighborhoods
0         M3A        North York                        Parkwoods
1         M4A        North York                 Victoria Village
2         M5A  Downtown Toronto                     Harbourfront
3         M6A        North York  Lawrence Heights,Lawrence Manor
5         M7A      Queen's Park                     Not assigned
6         M9A  Downtown Toronto                     Queen's Park
7         M1B       Scarborough                    Rouge,Malvern
9         M3B        North York                  Don Mills North
10        M4B         East York   Woodbine Gardens,Parkview Hill
12        M5B  Downtown Toronto          Ryerson,Garden District
Table shape: (103, 3)


In [13]:
df5['Neighborhoods'] = np.where((df5.Neighborhoods == 'Not assigned'),df5.Borough,df5.Neighborhoods)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
print(df5.head(10))
print('Table shape:',df5.shape)

   PostalCode           Borough                    Neighborhoods
0         M3A        North York                        Parkwoods
1         M4A        North York                 Victoria Village
2         M5A  Downtown Toronto                     Harbourfront
3         M6A        North York  Lawrence Heights,Lawrence Manor
5         M7A      Queen's Park                     Queen's Park
6         M9A  Downtown Toronto                     Queen's Park
7         M1B       Scarborough                    Rouge,Malvern
9         M3B        North York                  Don Mills North
10        M4B         East York   Woodbine Gardens,Parkview Hill
12        M5B  Downtown Toronto          Ryerson,Garden District
Table shape: (103, 3)


## latitude and the longitude coordinates of each neighborhood.

In [15]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df7 = pd.merge(df5, df6, on='PostalCode', how='right')

print(df7.head(10))


  PostalCode           Borough                    Neighborhoods   Latitude  \
0        M3A        North York                        Parkwoods  43.753259   
1        M4A        North York                 Victoria Village  43.725882   
2        M5A  Downtown Toronto                     Harbourfront  43.654260   
3        M6A        North York  Lawrence Heights,Lawrence Manor  43.718518   
4        M7A      Queen's Park                     Queen's Park  43.662301   
5        M9A  Downtown Toronto                     Queen's Park  43.667856   
6        M1B       Scarborough                    Rouge,Malvern  43.806686   
7        M3B        North York                  Don Mills North  43.745906   
8        M4B         East York   Woodbine Gardens,Parkview Hill  43.706397   
9        M5B  Downtown Toronto          Ryerson,Garden District  43.657162   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  
5 -79.532242  
6 -79.194353  
7 -79.352188  
8 -79.

## Maps to visualize your neighborhoods and how they cluster together.

In [41]:
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

CLIENT_ID = 'PH3C5IT4OPSSN3ZIBK0N4ZR5QXUDFJQZLUQOW2AU4M5CCVDB'
CLIENT_SECRET = 'DYAENG5F4ZLVSSIGN4FP4AIFSBBIJ2EPUQQUADO3E4SSA2X3'
VERSION = '20180605'
LIMIT = 100
radius = 500


#Neighborhood_name=df7.loc[0,'Neighborhoods']
lat=df7.loc[0,'Latitude']
lng=df7.loc[0,'Longitude']

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

results = requests.get(url).json()

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [48]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']


In [58]:
Neighborhood_name=df7.loc[5,'Neighborhoods']
lat=df7.loc[5,'Latitude']
lng=df7.loc[5,'Longitude']
print(Neighborhood_name,lat,lng)
results

Queen's Park 43.6678556 -79.53224240000002


{'meta': {'code': 200, 'requestId': '5dfb7440d03993001b50d44c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [65]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
filtered_columns=['venue.name','venue.categories','vanue.location.lat','vanue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()


TypeError: list indices must be integers or slices, not str